# Datenaufbereitung und Visualisierung Modularbeit

Author - Maximilian Hoffmann 2023

**Table of Contents**

- [Introduction](#introduction)
- [Hypothesis](#hypothesis)
- [Data](#data)
- [Data Preparation](#data-preparation)
- [Data Visualization](#data-visualization)
- [Data Analysis](#data-analysis)
- [Anonymization](#anonymization)
- [Conclusion](#conclusion)
- [References](#references)
- [Schlusswort](#schlusswort)

## Introduction

Für den Landtagswahlkampf Artikel soll ein Thema gefunden werden, welches relevanz für die Wähler und somit auch den Wahlkampf selbst hat. Dazu soll ein Datensatz gefunden werden, welcher die Relevanz des Themas belegt. Dieser Datensatz soll dann in einer Visualisierung dargestellt werden.

## Hypothesis

Da durch die Energiekostenexplosion im Jahr 2021 [<a id="reref1">[1](#ref1)], die Wähler sich mehr für das Thema Energie interessieren und für eine Energiewende einsetzen. Die Hypothese ist, dass es möglich ist allein durch Balkonkraftwerke die Energieversorgung eines Haushaltes zu decken und somit die Energiekosten zu senken. Diese Hypothese soll durch die Visualisierung belegt werden und Zeigen das eine Lockerung der Gesetze für Balkonkraftwerke sinnvoll ist. Das Ziel Klimaneutral zu werden würde so gefördert werden.

Weiterführend soll die Visualisierung zeigen, das durch die steigenden Energiekosten in den letzten Jahren Balkonkraftwerke zukünftig noch mehr an Bedeutung gewinnen werden. Ebenfalls soll gezeigt werden dass der Umstieg von fossilen Brennstoffen auf erneuerbare Energien sinnvoll ist und die abhängigkeit Deutschlands von anderen Ländern verringert.

Die Wahl des Balkonkraftwerks entstand da es sich um eine einfache und günstige Möglichkeit handelt, erneuerbare Energien zu nutzen. Hinzu ist es jedem Haushalt möglich, erneuerbare Energien zu nutzen und somit einen Beitrag zum Klimaschutz zu leisten.

---

Hierzu lassen sich einige zu klärende Fragen ableiten:

- Wie viel Energie kann ein Balkonkraftwerk erzeugen?
- Wie viel Energie verbraucht ein Haushalt?
- Wie viel Energie kann ein Haushalt durch ein Balkonkraftwerk erzeugen?
- Wie viel Energie kann ein Haushalt durch ein Balkonkraftwerk einsparen?
- Wie schnell amortisiert sich ein Balkonkraftwerk?

## Data


Die hierzu genutzten Datensätze des Datenarchivs [Open Data Bayern](https://opendata.bayern) sind:

- [Anteil Photovoltaik am Stromverbrauch (Gem.)](https://opendata.bayern/datasets/db272dfa-7dfb-45fb-9459-01840a3ce992?locale=de) - Bayerisches Landesamt für Umwelt
	- Dieser Datensatz wurde geählt da er zeigt welchen anteil Photovoltaik am Stromverbrauch hat.
- [Anteil erneuerbarer Energien am Stromverbrauch (Gem.)](https://opendata.bayern/datasets/9cebb88d-3877-4afe-a24c-3f24757f8257?locale=de) - Bayerisches Landesamt für Umwelt
	- Dieser Datensatz wurde geählt da er zeigt welchen anteil erneuerbare Energien am Stromverbrauch hat.
Zudem wurden Daten aus anderen Quellen genutzt:

- [Destatis](https://www.destatis.de/)
	- [Preisentwicklungen](https://www.destatis.de/DE/Presse/Pressemitteilungen/2022/03/PD22_N016_61.html)
- [Bayerisches Staatsministerium für Wirtschaft, Landesentwicklung und Energie](https://www.stmwi.bayern.de)
	- [bayerische Energiebilanz 2020](https://www.stmwi.bayern.de/energie/energiedaten/energiebilanz-2020/)

Selbst erhobene Daten aus Balkonkraftwerk-Testanlage in München (siehe [Anonymization](#anonymization)):

- Die CSV-Datei wurde über die Messung eines zwischengeschalteten Shelly-Plus-1PM (WLAN-Schalter mit Strommessfunktion) erhoben. Der export der Daten erfolgte über [HomeAssistant](https://www.home-assistant.io).
	- Spezifikationen der Balkonkraftwerk-Testanlage:
		- 2 Platten mit je 310Watt
		- Wecheslrichter mit 600Watt
		- 1 Shelly-Plus-1PM

## Data Preparation

Zur vorbereitung der Daten wurden die verschiedensten Datentypen aus `src/data_raw` manuel in ein nutzbares Format (z.B. CSV) gebracht. Hierzu wurden [Excel](https://www.microsoft.com/en-us/microsoft-365/excel) und [OpenRefine](https://openrefine.org) genutzt. Nach der Bearbeitung wurden die Daten in `src/data` gespeichert. 

Bei Begegnung sensibler Daten wurde die [Anonymisierung](#anonymization) durchgeführt und in beiden Ordnern übernommen. Wenn Daten schon im passenden Format vorlagen, wurden diese direkt in `src/data` gespeichert.

Hin und wieder begegnem einem auch fehlende Daten. Diese werden im Code auch als solche gehandhabt. Im falle der Kartendaten schränkt dies die Visualisierung ein, da die Daten nicht vollständig sind. Hier gab es leider keine Möglichkeit die Daten zu vervollständigen und somit wurde die Visualisierung auf die vorhandenen Daten beschränkt, wenn mit fehlenden Daten versucht wird zu arbeiten wird jedoch versucht einen der breiter gefächerten Datensätze zu nutzen.

## Data Visualization

Für die Datenvisualisierung werden folgende Bibliotheken genutzt:


- [Matplotlib](https://matplotlib.org/) for plotting
- [Pandas](https://pandas.pydata.org/) for data manipulation
- [Numpy](https://numpy.org/) for data manipulation
- [Plotly](https://plotly.com/python/) for interactive plots
- [ipywidgets](https://ipywidgets.readthedocs.io/en/latest/) for user inputs / interactive plots

<!-- TODO: DELETE BELOW -->

- [Seaborn](https://seaborn.pydata.org/) for plotting
- [Pyproj](https://pyproj4.github.io/pyproj/stable/) for coordinate transformations
- [sklearn](https://scikit-learn.org/stable/) for plot prediction


In [1]:
# importing the required libraries
import matplotlib.pyplot as plt
import pandas as pd
import ipywidgets as widgets

import folium
from owslib.wms import WebMapService

In [2]:
#Hilfsfunktionen

def get_data_from_wms(url, layer_name, lat, lon):
    #hier wird ein möglichst kleiner Bereich um den Marker herum abgefragt
    minx = lon -0.00001
    maxx = lon +0.00001
    miny = lat -0.00001
    maxy = lat +0.00001
    width = 1
    height = 1
    x = width / 2
    y = height / 2
    
    wms = WebMapService(url)
    response = wms.getfeatureinfo(
        layers=[layer_name],
        format='application/json',
        bbox=(minx, miny, maxx, maxy),
        size=(width, height),
        srs='EPSG:4326',
        query_layers=[layer_name],
        xy=(x, y),
        info_format='application/json'
    )

    data = response.read()
    return data

def create_map(wms_url, layer_name, select_lat, select_lon, header):
    germany_location = [48.99, 11.240785]
    select_location = [select_lat, select_lon]
    
    #hier wird die Karte erstellt
    m = folium.Map(germany_location, zoom_start=8, dragging=False, zoom_control=False, scrollWheelZoom=False, doubleClickZoom=False, touchZoom=False, width='80%')
    
    #hier wird der WMS Layer hinzugefügt
    folium.LatLngPopup().add_to(m)
    folium.WmsTileLayer(
        url=wms_url,
        layers=layer_name,
        name=layer_name,
        fmt='image/png',
        transparent=True
    ).add_to(m)

    #hier wird der Marker gesetzt
    movemarker = folium.Marker(select_location, tooltip=None, popup=None, draggable=False)
    m.add_child(movemarker)

    display(header, m)
    
#zum auslesen der Daten aus dem XML
def parse_xml(data, dataname):
	import xml.etree.ElementTree as ET
	root = ET.fromstring(data)
	fields_element = root.find('.//{http://www.esri.com/wms}FIELDS')
	rdata = fields_element.get(dataname)
	return rdata

In [3]:
#nun zur eigentlichen Visualisierung

#diese funktion wird aufgerufen, wenn die interaktive Karte erstellt oder verändert wird
def interact_pv_gem(select_lat, select_lon, month, panel_watt, price_balkonkraftwerk, price_kwh, verbrauch_kwh_pro_jahr, efficiency, panel_area):
    #karte für den Gesammtverbrauch einer Gemeinde
    wms_verbrauch=WebMapService('https://www.lfu.bayern.de/gdi/wms/energieatlas/statistik_photovoltaik?REQUEST=GetCapabilities&SERVICE=WMS').url
    create_map(wms_url=wms_verbrauch, layer_name='anteil_pv_gem', select_lat=select_lat, select_lon=select_lon, 
        header="Diese Karte zeigt den Anteil des Gesamtverbrauchs einer Gemeinde, der durch erneuerbare Energien (z.B. Photovoltaic) gedeckt wird. Der Anteil wird in Prozent angegeben.")
    
    #karte für die Globalstrahlung
    wms_globalstrahlung=WebMapService('https://wirtschaft-risby.bayern.de/RisGate/servlet/Globalstrahlung').url
    def month_from_number(int):
        months = ['januar', 'februar', 'maerz', 'april', 'mai', 'juni', 'juli', 'august', 'september', 'oktober', 'november', 'dezember', 'jahr']
        return months[int-1]
    strahlung_zeit = 'Globalstrahlung_' + month_from_number(month)
    create_map(wms_url=wms_globalstrahlung, layer_name=strahlung_zeit, select_lat=select_lat, select_lon=select_lon,
        header="Diese Karte zeigt die Globalstrahlung in Bayern. Die Globalstrahlung ist die Summe aus direkter und diffuser Strahlung. Die Globalstrahlung wird in kWh/m² angegeben.")
        
    kwh_year_range = parse_xml(get_data_from_wms(wms_globalstrahlung, strahlung_zeit, select_lat, select_lon), 'radiation_class').split('-')
    kwh_per_meter_per_year = (int(kwh_year_range[0]) + int(kwh_year_range[1])) / 2
    if month != 13: kwh_per_meter_per_year = kwh_per_meter_per_year * 12
    panel_kw = panel_watt / 1000
    production_per_year = panel_kw * panel_area * efficiency * kwh_per_meter_per_year
    cost_per_year = verbrauch_kwh_pro_jahr * price_kwh
    savings_per_year = production_per_year * price_kwh
    if savings_per_year > cost_per_year: savings_per_year = cost_per_year #sicherstellen, dass die Einsparungen nicht höher sind als die uhrsprünglichen Kosten
    
    
    
    
    
print("Hier können auch andere Werte eingegeben werden. Die Werte werden dann in der Karte angezeigt.")

#hier wird der Interaktive Teil erstellt
widgets.interact(interact_pv_gem,
    select_lat=widgets.FloatText(value='48.15451802076226', description='Longitude:', disabled=False, continuous_update=False, style={'description_width': 'initial'}, layout=widgets.Layout(width='fit-content')),
    select_lon=widgets.FloatText(value='11.556412766827073', description='Latitude: ', disabled=False, continuous_update=False, style={'description_width': 'initial'}, layout=widgets.Layout(width='fit-content')),
    month=widgets.IntSlider(min=1, max=13, step=1, value=13, description='Month (13 for whole year):', continuous_update=False, style={'description_width': 'initial'}, layout=widgets.Layout(width='40%')),
    panel_watt=widgets.FloatText(value=600, description='Panel Watt:', disabled=False, continuous_update=False, style={'description_width': 'initial'}, layout=widgets.Layout(width='fit-content')),
    price_balkonkraftwerk=widgets.FloatText(value=1070, description='Balkonkraftwerk Preis insgesammt in €:', disabled=False, continuous_update=False, style={'description_width': 'initial'}, layout=widgets.Layout(width='fit-content')),
    price_kwh=widgets.FloatText(value=0.3, description='Price per kWh in €:', disabled=False, continuous_update=False, style={'description_width': 'initial'}, layout=widgets.Layout(width='fit-content')),
    verbrauch_kwh_pro_jahr=widgets.FloatText(value=3000, description='Verbrauch kWh pro Jahr:', disabled=False, continuous_update=False, style={'description_width': 'initial'}, layout=widgets.Layout(width='fit-content')),
    efficiency=widgets.FloatSlider(value=0.25, min=0, max=1, step=0.01, description='Efficiency:', disabled=False, continuous_update=False, style={'description_width': 'initial'}, layout=widgets.Layout(width='40%')),
    panel_area=widgets.FloatText(value=3, description='Panel Area per Panel in m^2:', disabled=False, continuous_update=False, style={'description_width': 'initial'}, layout=widgets.Layout(width='fit-content'))
)

Hier können auch andere Werte eingegeben werden. Die Werte werden dann in der Karte angezeigt.


interactive(children=(FloatText(value=48.15451802076226, description='Longitude:', layout=Layout(width='fit-co…

<function __main__.interact_pv_gem(select_lat, select_lon, month, panel_watt, price_balkonkraftwerk, price_kwh, verbrauch_kwh_pro_jahr, efficiency, panel_area)>

## Data Analysis

**Wie viel Energie kann ein Balkonkraftwerk erzeugen?**
Hierzu nutzen wir die aus der Testanlage gewonnenen Daten.

---

**Wie viel Energie verbraucht ein Haushalt?**
Hierzu werden Durchschnittsdaten aus dem [Bayerischen Staatsministerium für Wirtschaft, Landesentwicklung und Energie](https://www.stmwi.bayern.de) genutzt.

---

**Wie viel Energie kann ein Haushalt durch ein Balkonkraftwerk erzeugen?**
Hierzu nutzen wir die aus der Testanlage gewonnenen Daten, sowie Erwartungswerte gewonnen aus den [Open Data Bayern](https://opendata.bayern) Datensätzen.
Die Erwartungswerte werden sowohl als Durchschnitt als auch für die Monate berechnet. Hierzu werden Historische Daten genutzt.

---

**Wie viel Stromkosten kann ein Haushalt durch ein Balkonkraftwerk einsparen?**
Um die eingesparten Stromkosten zu berechnen wird die Differenz zwischen dem Stromverbrauch und der Stromerzeugung durch das Balkonkraftwerk berechnet. Hierzu werden sowohl die Daten aus der Testanlage genutzt als auch Strompreisdaten.

---

**Wie schnell amortisiert sich ein Balkonkraftwerk?**
Hierzu wird berechnet wie viel für das Balkonkraftwerk ausgegeben werden muss und wie viel durch das Balkonkraftwerk eingespart wird. Hierzu werden die Daten aus der Testanlage genutzt, sowie Strompreisdaten.


## Anonymization

Wie schon in der [Datenvorbereitung](#data-preparation) erwähnt, wurden die Daten mit sensiblen Informationen anonymisiert.

Da die in dieser Arbeit genutzten Daten aus öffentlichen Quellen stammen, wurden größtenteils nicht anonymisiert. Die Daten aus der Balkonkraftwerk-Testanlage in München wurden jedoch anonymisiert. Dies wurde durch das entfernen der IP-Adresse und des Standortes erreicht. Die Daten wurden sowohl in `src/data_raw` als auch in `src/data` anonymisiert.

Der Grund für die Anonymisierung in diesem Fall der Datenschutz und die Privatsphäre der Nutzer. Die Daten wurden von einem privaten Haushalt erhoben und könnten somit Rückschlüsse auf den Haushalt zulassen. Dies soll durch die Anonymisierung verhindert werden.

## Conclusion

Die oben visualisierten Daten belegen, # TODO: FILL




**Anmerkung:** 
*Hierzu sei zu beachten das einige Felder und Schieberegler bearbeitet werden können, um die Visualisierung zu verändern. Dies ist durch die Nutzung von [ipywidgets](https://ipywidgets.readthedocs.io/en/latest/) möglich. Die Visualisierung ist interaktiv und kann somit vom Nutzer verändert werden. Jegliche Schlussfolgerung die aus der Visualisierung gezogen werden beziehen sich auf die Standardwerte der Schieberegler. Dies dient dem zweck das der Nutzer hier, nicht nur aber auch, seinen eigenen Strompreis oder seinen eigenen Stromverbrauch eintragen kann und somit die Visualisierung für sich anpassen kann.*

## References

- [<a id="ref1">[1](#reref1)] : [Energiepreisentwicklung](https://www.destatis.de/DE/Presse/Pressemitteilungen/2022/03/PD22_N016_61.html)
- [<a id="ref2">[2](#reref2)] : []()

## Schlusswort

Das Projekt hat echt Spass gemacht, auch wenn die Themasuche etwas länger gedauert hat. Ich habe mich für das Thema Balkonkraftwerk entschieden da ich selbst an der Testanlage beteiligt bin und mich das Thema interessiert. Ehrlich gesagt freue ich mich schon drauf eine ähnliche Datenvisualisierung in mein Home-Assistant/Grafana Dashboard einzubauen wenn die Klausurenphase vorbei ist.

Vielleicht überzeug das Projekt ja nicht nur als Modularbeit zum Landtagswahlkampd sondern auch sich Gedanken über ein Balkonkraftwerk zu machen. Man denke nur an die ganzen Balkone die in München bisher ungenutzt bleiben. 

Die Datei `src/testing.ipynb` hab ich extra drin gelassen da ich diese für die Entwicklung genutzt habe und sie mir als Dokumentation dient. Eventuell versteckt sich ja der ein oder andere interessante Code-Schnipsel darin. 